In [11]:
import numpy as np

def variogram_model(h, model="linear", sill=1.0, range_param=1.0):
    """
    Variogram model function.
    :param h: Distance (lag)
    :param model: Type of variogram ('linear', 'exponential', 'gaussian')
    :param sill: Variance at large distances
    :param range_param: Range parameter
    :return: Variogram value
    """
    if model == "linear":
        return np.minimum(sill, h / range_param * sill)
    elif model == "exponential":
        return sill * (1 - np.exp(-h / range_param))
    elif model == "gaussian":
        return sill * (1 - np.exp(-(h / range_param) ** 2))
    else:
        raise ValueError("Unsupported variogram model.")

def compute_covariance_matrix(coords, variogram_func):
    """
    Compute the covariance matrix for given coordinates.
    :param coords: Array of coordinates (n_samples, 2)
    :param variogram_func: Variogram function
    :return: Covariance matrix
    """
    n = len(coords)
    cov_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            h = np.linalg.norm(coords[i] - coords[j])  # Euclidean distance
            cov_matrix[i, j] = variogram_func(0) - variogram_func(h)  # Covariance
    return cov_matrix

def kriging(primary_coords, primary_values, target_coords, variogram_func):
    """
    Perform ordinary kriging.
    :param primary_coords: Coordinates of primary data points
    :param primary_values: Values of primary data points
    :param target_coords: Coordinates where estimation is required
    :param variogram_func: Variogram function
    :return: Estimated values at target coordinates
    """
    n = len(primary_coords)
    
    # Compute covariance matrix and cross-covariance vector
    cov_matrix = compute_covariance_matrix(primary_coords, variogram_func)
    cov_vector = np.array([variogram_func(0) - variogram_func(np.linalg.norm(primary_coords[i] - target_coords)) for i in range(n)])
    
    # Solve kriging system
    weights = np.linalg.solve(cov_matrix, cov_vector)
    
    # Estimate value at target location
    estimated_value = np.dot(weights, primary_values)
    
    return estimated_value

def cokriging(primary_coords, primary_values, secondary_coords, secondary_values, target_coords, primary_variogram_func, secondary_variogram_func, cross_variogram_func):
    """
    Perform ordinary cokriging.
    :param primary_coords: Coordinates of primary data points
    :param primary_values: Values of primary data points
    :param secondary_coords: Coordinates of secondary data points
    :param secondary_values: Values of secondary data points
    :param target_coords: Coordinates where estimation is required
    :param primary_variogram_func: Variogram function for primary variable
    :param secondary_variogram_func: Variogram function for secondary variable
    :param cross_variogram_func: Cross-variogram function between primary and secondary variables
    :return: Estimated values at target coordinates
    """
    
    n1 = len(primary_coords)
    n2 = len(secondary_coords)
    
    # Compute covariance matrices and cross-covariance vectors
    cov_matrix_primary = compute_covariance_matrix(primary_coords, primary_variogram_func)
    
    cov_matrix_secondary = compute_covariance_matrix(secondary_coords, secondary_variogram_func)
    
    cross_cov_matrix = np.zeros((n1 + n2, n1 + n2))
    
    # Fill cross-covariance matrix (primary-secondary interaction)
    for i in range(n1):
        for j in range(n2):
            h = np.linalg.norm(primary_coords[i] - secondary_coords[j])
            cross_cov_matrix[i, j + n1] = cross_variogram_func(0) - cross_variogram_func(h)
            cross_cov_matrix[j + n1, i] = cross_cov_matrix[i, j + n1]

    # Combine primary and secondary covariance matrices into a single block matrix
    block_cov_matrix = np.zeros((n1 + n2, n1 + n2))
    block_cov_matrix[:n1, :n1] = cov_matrix_primary
    block_cov_matrix[n1:, n1:] = cov_matrix_secondary
    block_cov_matrix[:n1, n1:] = cross_cov_matrix[:n1, n1:]
    block_cov_matrix[n1:, :n1] = cross_cov_matrix[n1:, :n1]

    # Compute cross-covariance vector for target location
    cov_vector_primary = np.array([primary_variogram_func(0) - primary_variogram_func(np.linalg.norm(primary_coords[i] - target_coords)) for i in range(n1)])
    cov_vector_secondary = np.array([secondary_variogram_func(0) - secondary_variogram_func(np.linalg.norm(secondary_coords[i] - target_coords)) for i in range(n2)])
    
    cov_vector_combined = np.concatenate([cov_vector_primary, cov_vector_secondary])

    # Solve cokriging system
    weights_combined = np.linalg.solve(block_cov_matrix, cov_vector_combined)

    # Separate weights for primary and secondary variables
    weights_primary = weights_combined[:n1]
    weights_secondary = weights_combined[n1:]

    # Estimate value at target location
    estimated_value = np.dot(weights_primary, primary_values) + np.dot(weights_secondary, secondary_values)

    return estimated_value

# Example Usage
if __name__ == "__main__":
    # Primary variable data (coordinates and values)
    primary_coords = np.array([[0.0, 0.0], [0.5, 0.5], [1.0, 0.0], [1.5, 0.5]])
    primary_values = np.array([10.0, 12.5, 15.0, 17.5])

    # Secondary variable data (coordinates and values)
    secondary_coords = np.array([[0.25, 0.25], [0.75, 0.75], [1.25, 0.25], [1.75, 0.75]])
    secondary_values = np.array([20.0, 22.5, 25.0, 27.5])

    # Target location (where we want to estimate values)
    target_coords = np.array([0.75, 0.25])

    # Define variogram models
    sill_primary = 10.0
    range_primary = 2.0
    sill_secondary = 20.0
    range_secondary = 2.5

    def primary_variogram(h):
        return variogram_model(h, model="linear", sill=sill_primary, range_param=range_primary)

    def secondary_variogram(h):
        return variogram_model(h, model="linear", sill=sill_secondary, range_param=range_secondary)

    def cross_variogram(h):
        return variogram_model(h, model="linear", sill=min(sill_primary, sill_secondary), range_param=max(range_primary, range_secondary))

    # Perform kriging
    kriging_estimate = kriging(primary_coords, primary_values, target_coords, primary_variogram)
    
    print(f"Kriging Estimate at {target_coords}: {kriging_estimate}")

    # Perform cokriging
    cokriging_estimate = cokriging(
        primary_coords,
        primary_values,
        secondary_coords,
        secondary_values,
        target_coords,
        primary_variogram,
        secondary_variogram,
        cross_variogram,
    )
    
    print(f"Cokriging Estimate at {target_coords}: {cokriging_estimate}")


Kriging Estimate at [0.75 0.25]: 12.785698126683647
Cokriging Estimate at [0.75 0.25]: 22.47054677449223
